In [169]:
import datetime
import pandas as pd
import numpy as np



In [170]:
from alpha_vantage.timeseries import TimeSeries
alphavantage_api_key = 'LHSF13XCYF6MXI3J'
ts = TimeSeries(key= alphavantage_api_key, output_format='pandas')



## Function: Get Stock Price from AlphaVantage

In [171]:
def get_price_array(ticker):
    try:
        data, meta_data = ts.get_daily(symbol=ticker, outputsize='compact')
        data = data.reset_index()
        data['Ticker'] = ticker
        return data
    except:
        pass

## Load Oil Sentiment Data

In [172]:
csv_path = "Oil_Company_Tweet_Data_Latest.csv"
df = pd.read_csv(csv_path, encoding='ISO-8859-1')

df.head()

,Unnamed: 0,Actual Date,Date,Media Sources,Negative,Neutral,Positive,Source Account,Tweet Polarity,Tweets Ago
0,0,Fri Jan 12 11:27:15 +0000 2018,2018-01-12,@OPECnews,0.0,1.000,0.000,Atlantic Council,0.0000,1
1,1,Fri Jan 12 05:30:53 +0000 2018,2018-01-12,@OPECnews,0.0,0.769,0.231,wPOWER Hub,0.5994,2
2,2,Fri Jan 12 05:27:34 +0000 2018,2018-01-12,@OPECnews,0.0,0.719,0.281,Energy 4 Impact,0.5994,3
3,3,Thu Jan 11 14:11:58 +0000 2018,2018-01-11,@OPECnews,0.0,1.000,0.000,'THE STRANGER ðð' I AM NIBIRU! âð´ð',0.0000,4
4,4,Thu Jan 11 06:05:35 +0000 2018,2018-01-11,@OPECnews,0.0,0.812,0.188,aramco | Ø£Ø±Ø§Ù ÙÙ,0.5267,5


In [173]:
df.columns

Index(['Unnamed: 0', 'Actual Date', 'Date', 'Media Sources', 'Negative',
       'Neutral', 'Positive', 'Source Account', 'Tweet Polarity',
       'Tweets Ago'],
      dtype='object')

## Map Media Source to Ticker to later Retrieve Stock Price

In [174]:
oil_ticker = {'@Shell': 'RDS.A ',
             '@OPECnews': 'USO',
              '@exxonmobil': 'XOM',
             '@Chevron': 'CVX',
              '@conocophillips': 'COP',
             '@OXY_Petroleum': 'OXY',
              '@ValeroEnergy': 'VLO',
             '@Phillips66Co': 'PSX',
              '@MarathonPetroCo': 'MPC',
             '@DevonEnergy':'DVN'}

## Get Stock Ticker in order to retrieve Price

In [175]:
df['Ticker'] = [oil_ticker[x] for x in df['Media Sources']]

In [176]:
ticker_list = list(df.Ticker.unique())
ticker_list

['USO', 'RDS.A ', 'XOM', 'CVX', 'COP', 'OXY', 'VLO', 'PSX', 'MPC', 'DVN']

## Generate Stock Prices Chart

In [177]:
for ticker in ticker_list:
    temp_df = (get_price_array(ticker))
    stock_df = stock_df.append(temp_df, ignore_index=True)


In [178]:
stock_df.columns

Index(['1. open', '2. high', '3. low', '4. close', '5. volume', 'Price',
       'Ticker', 'date'],
      dtype='object')

In [179]:
stock_df = stock_df.loc[:,['date', 'Ticker', '4. close']]

In [180]:
stock_df = stock_df.rename(columns={'4. close':'Price'})

In [182]:
len(stock_df)

7300

## To be Deleted

In [183]:
grouped_df = df.groupby(['Date', 
                        'Tweet Polarity', 
                        'Ticker'])

In [184]:
mean_df = grouped_df['Tweet Polarity'].mean()

In [185]:
oil_df = pd.DataFrame(mean_df)
oil_df = oil_df.drop('Tweet Polarity', axis=1)
oil_df = oil_df.reset_index()

In [186]:
oil_df.head()

,Date,Tweet Polarity,Ticker
0,2018-01-02,-0.2500,MPC
1,2018-01-03,-0.4215,PSX
2,2018-01-03,-0.2500,MPC
3,2018-01-03,0.0000,MPC
4,2018-01-03,0.7269,MPC


In [187]:
oil_df.tail()

,Date,Tweet Polarity,Ticker
166,2018-01-13,0.6249,XOM
167,2018-01-13,0.7003,CVX
168,2018-01-13,0.7096,PSX
169,2018-01-13,0.7184,CVX
170,2018-01-13,0.7772,CVX


## Get Stock Price for Oil Sentiment (Ticker, Date)

In [192]:
oil_sentiment_df = pd.merge(oil_df, stock_df, left_on=['Date', 'Ticker'], right_on=['date', 'Ticker'], how='inner')

In [194]:
oil_sentiment_df.tail()

,Date,Tweet Polarity,Ticker,date,Price
1035,2018-01-12,0.4574,MPC,2018-01-12,NaN
1036,2018-01-12,0.4574,MPC,2018-01-12,NaN
1037,2018-01-12,0.4574,MPC,2018-01-12,NaN
1038,2018-01-12,0.4574,MPC,2018-01-12,NaN
1039,2018-01-12,0.4574,MPC,2018-01-12,71.42


In [198]:
oil_sentiment_df = oil_sentiment_df.dropna()

In [196]:
oil_sentiment_df.to_csv('oil_sentiment.csv', sep=',')

In [202]:
oil_sentiment_df = oil_sentiment_df.sort_values(by=['Ticker', 'Date'], ascending=[True, True], inplace=False)

In [203]:
oil_sentiment_df

,Date,Tweet Polarity,Ticker,date,Price
295,2018-01-10,-0.5859,COP,2018-01-10,57.65
303,2018-01-10,-0.2960,COP,2018-01-10,57.65
311,2018-01-10,-0.2023,COP,2018-01-10,57.65
319,2018-01-10,0.0000,COP,2018-01-10,57.65
327,2018-01-10,0.3436,COP,2018-01-10,57.65
335,2018-01-10,0.5106,COP,2018-01-10,57.65
343,2018-01-10,0.6800,COP,2018-01-10,57.65
463,2018-01-11,-0.6486,COP,2018-01-11,59.18
471,2018-01-11,-0.6369,COP,2018-01-11,59.18
479,2018-01-11,-0.5423,COP,2018-01-11,59.18
